In [15]:
import itertools
import pulp

def find_multiset(n, v_dict, m):
    """
    Solve for a multiset M in F2^n satisfying divisibility properties:
      - For each beta in F2^n \ {0}, sum_x (-1)^{<beta,x>} * mu[x] = 2^{v_beta+1} * d_beta
      - sum_x mu[x] = 2^m
    Additionally, enforce each d_beta to be an odd integer.

    Args:
        n (int): dimension of the F2 vector space
        v_dict (dict[int, int]): map from beta (encoded as int 1..2^n-1) to v_beta
        m (int): exponent for total multiset size (|M| = 2^m)

    Returns:
        (dict[int,int], dict[int,int]): solutions for mu(x) and d_beta
    """
    
    
    # Create MILP problem: minimize max multiplicity for uniformity
    prob = pulp.LpProblem("Multiset_Divisibility_OddDBeta", pulp.LpMinimize)

    # Variables: mu[x] >= 0 integer multiplicities
    mu = {x: pulp.LpVariable(f"mu_{x}", lowBound=0, cat='Integer')
          for x in range(2**n)}
    # d_beta integer (no bounds)
    d = {beta: pulp.LpVariable(f"d_{beta}", lowBound=None, cat='Integer')
         for beta in range(1, 2**n)}

    # Constraint: total size = 2^m
    prob += sum(mu[x] for x in mu) == 2**m, "TotalSize"

#     # Constraints per beta != 0
#     for beta in range(1, 2**n):
#         v = v_dict.get(beta, 0)
#         # Divisibility: sum_x (-1)^{<beta,x>} mu[x] = 2^{v+1} * d_beta
#         expr = pulp.lpSum(((-1)**(bin(x & beta).count('1') & 1)) * mu[x]
#                           for x in mu)
#         prob += expr == 2**(v + 1) * (2 * d[beta] + 1), f"Divisible_beta_{beta}"

    # Constraints per beta != 0
    for beta in range(1,2**n):
        v = v_dict.get(beta, 0)
        # Divisibility: sum_x (-1)^{<beta,x>} mu[x] = 2^{v+1} * d_beta
        expr = pulp.lpSum(((-1)**(bin(x & beta).bit_count() & 1)) * mu[x]
                          for x in mu)
        prob += expr == 2**(v + 1) * (2 * d[beta] + 1), f"Divisible_beta_{beta}"
    
     # Auxiliary for uniformity: minimize the maximum mu[x]
#     T = pulp.LpVariable("T", lowBound=0, cat='Integer')
#     for x in mu:
#         prob += mu[x] <= T
#     prob += T, "Minimize_max_mu"
    print(prob)
    # Optional objective: find any feasible solution
    prob += 0, "Objective"

    # Solve with GLPK (or swap to CBC with path if desired)
    result = prob.solve(pulp.GLPK_CMD(msg=True))
    if pulp.LpStatus[result] != 'Optimal':
        raise ValueError(f"No optimal solution found: status {pulp.LpStatus[result]}")

    # Extract solutions
    mu_sol = {x: int(mu[x].value()) for x in mu}
    d_sol = {beta: int(d[beta].value()) for beta in v_dict.keys()}
    return mu_sol, d_sol


if __name__ == "__main__":
    # Example usage for n=4
    n = 4
    # Specify some v_beta values
    v_dict = {1: 1, 2:1}
    m = 5  # total size = 2^4 = 16

    mu_solution, d_solution = find_multiset(n, v_dict, m)
    print("Mu(x) values:", mu_solution)
    print("Odd d_beta values:", d_solution)

Multiset_Divisibility_OddDBeta:
MINIMIZE
None
SUBJECT TO
TotalSize: mu_0 + mu_1 + mu_10 + mu_11 + mu_12 + mu_13 + mu_14 + mu_15 + mu_2
 + mu_3 + mu_4 + mu_5 + mu_6 + mu_7 + mu_8 + mu_9 = 32

Divisible_beta_1: - 8 d_1 + mu_0 - mu_1 + mu_10 - mu_11 + mu_12 - mu_13
 + mu_14 - mu_15 + mu_2 - mu_3 + mu_4 - mu_5 + mu_6 - mu_7 + mu_8 - mu_9 = 4

Divisible_beta_2: - 8 d_2 + mu_0 + mu_1 - mu_10 - mu_11 + mu_12 + mu_13
 - mu_14 - mu_15 - mu_2 - mu_3 + mu_4 + mu_5 - mu_6 - mu_7 + mu_8 + mu_9 = 4

Divisible_beta_3: - 4 d_3 + mu_0 - mu_1 - mu_10 + mu_11 + mu_12 - mu_13
 - mu_14 + mu_15 - mu_2 + mu_3 + mu_4 - mu_5 - mu_6 + mu_7 + mu_8 - mu_9 = 2

Divisible_beta_4: - 4 d_4 + mu_0 + mu_1 + mu_10 + mu_11 - mu_12 - mu_13
 - mu_14 - mu_15 + mu_2 + mu_3 - mu_4 - mu_5 - mu_6 - mu_7 + mu_8 + mu_9 = 2

Divisible_beta_5: - 4 d_5 + mu_0 - mu_1 + mu_10 - mu_11 - mu_12 + mu_13
 - mu_14 + mu_15 + mu_2 - mu_3 - mu_4 + mu_5 - mu_6 + mu_7 + mu_8 - mu_9 = 2

Divisible_beta_6: - 4 d_6 + mu_0 + mu_1 - mu_10 - mu_11 - m

+2117314: mip =     not found yet >=   0.000000000e+00        (51273; 211639)
+2126297: mip =     not found yet >=   0.000000000e+00        (51462; 212929)
+2137748: mip =     not found yet >=   0.000000000e+00        (51917; 214297)
+2150824: mip =     not found yet >=   0.000000000e+00        (52522; 215455)
+2162423: mip =     not found yet >=   0.000000000e+00        (52988; 216641)
+2174627: mip =     not found yet >=   0.000000000e+00        (53669; 217708)
+2185915: mip =     not found yet >=   0.000000000e+00        (54430; 218846)
+2197413: mip =     not found yet >=   0.000000000e+00        (55148; 219936)
Time used: 300.0 secs.  Memory used: 88.1 Mb.
+2208716: mip =     not found yet >=   0.000000000e+00        (55411; 221062)
+2217343: mip =     not found yet >=   0.000000000e+00        (55773; 222269)
+2228537: mip =     not found yet >=   0.000000000e+00        (56450; 223366)
+2237183: mip =     not found yet >=   0.000000000e+00        (56826; 224621)
+2245211: mip =   

+2941819: mip =     not found yet >=   0.000000000e+00        (83732; 305295)
+2947117: mip =     not found yet >=   0.000000000e+00        (83841; 306082)
+2953191: mip =     not found yet >=   0.000000000e+00        (84011; 306902)
Time used: 780.0 secs.  Memory used: 128.1 Mb.
+2959375: mip =     not found yet >=   0.000000000e+00        (84218; 307642)
+2964932: mip =     not found yet >=   0.000000000e+00        (84402; 308333)
+2971734: mip =     not found yet >=   0.000000000e+00        (84690; 309031)
+2979199: mip =     not found yet >=   0.000000000e+00        (85061; 309695)
+2986486: mip =     not found yet >=   0.000000000e+00        (85501; 310325)
+2994524: mip =     not found yet >=   0.000000000e+00        (85856; 311004)
+3001202: mip =     not found yet >=   0.000000000e+00        (86177; 311667)
+3008206: mip =     not found yet >=   0.000000000e+00        (86737; 312304)
+3013910: mip =     not found yet >=   0.000000000e+00        (86979; 312986)
+3021246: mip =  

+3477251: mip =     not found yet >=   0.000000000e+00        (105486; 366843)
+3482445: mip =     not found yet >=   0.000000000e+00        (105721; 367431)
+3487400: mip =     not found yet >=   0.000000000e+00        (105920; 368030)
+3492050: mip =     not found yet >=   0.000000000e+00        (105982; 368668)
+3497046: mip =     not found yet >=   0.000000000e+00        (106130; 369275)
+3501350: mip =     not found yet >=   0.000000000e+00        (106182; 369892)
+3505680: mip =     not found yet >=   0.000000000e+00        (106282; 370575)
+3509695: mip =     not found yet >=   0.000000000e+00        (106294; 371193)
+3514860: mip =     not found yet >=   0.000000000e+00        (106445; 371763)
+3520256: mip =     not found yet >=   0.000000000e+00        (106584; 372338)
+3525016: mip =     not found yet >=   0.000000000e+00        (106689; 372942)
Time used: 1320.1 secs.  Memory used: 165.3 Mb.
+3529352: mip =     not found yet >=   0.000000000e+00        (106788; 373546)
+353

+3943999: mip =     not found yet >=   0.000000000e+00        (120771; 421611)
+3949890: mip =     not found yet >=   0.000000000e+00        (121595; 421929)
+3955535: mip =     not found yet >=   0.000000000e+00        (122341; 422255)
+3960547: mip =     not found yet >=   0.000000000e+00        (123011; 422581)
+3965131: mip =     not found yet >=   0.000000000e+00        (123608; 422906)
+3970493: mip =     not found yet >=   0.000000000e+00        (123966; 423285)
+3974464: mip =     not found yet >=   0.000000000e+00        (123983; 423773)
+3979466: mip =     not found yet >=   0.000000000e+00        (124093; 424263)
Time used: 1800.1 secs.  Memory used: 188.7 Mb.
+3984002: mip =     not found yet >=   0.000000000e+00        (124125; 424723)
+3989594: mip =     not found yet >=   0.000000000e+00        (124321; 425167)
+3995355: mip =     not found yet >=   0.000000000e+00        (124471; 425683)
+4000145: mip =     not found yet >=   0.000000000e+00        (124604; 426141)
+400

...

Time used: 19084.9 secs.  Memory used: 507.0 Mb.
+10835947: mip =     not found yet >=   0.000000000e+00        (326026; 1278585)
+10837515: mip =     not found yet >=   0.000000000e+00        (326049; 1278742)
+10838690: mip =     not found yet >=   0.000000000e+00        (326052; 1278914)
+10839733: mip =     not found yet >=   0.000000000e+00        (326037; 1279043)
+10841124: mip =     not found yet >=   0.000000000e+00        (326033; 1279240)
+10843188: mip =     not found yet >=   0.000000000e+00        (326032; 1279454)
+10844741: mip =     not found yet >=   0.000000000e+00        (326066; 1279615)
+10846043: mip =     not found yet >=   0.000000000e+00        (326070; 1279820)
+10847664: mip =     not found yet >=   0.000000000e+00        (326105; 1279998)
+10849470: mip =     not found yet >=   0.000000000e+00        (326132; 1280208)
+10851506: mip =     not found yet >=   0.000000000e+00        (326185; 1280366)
+10853003: mip =     not found yet >=   0.000000000e+00     

+10976404: mip =     not found yet >=   0.000000000e+00        (329005; 1295058)
+10977570: mip =     not found yet >=   0.000000000e+00        (329041; 1295232)
+10979127: mip =     not found yet >=   0.000000000e+00        (329111; 1295407)
+10980654: mip =     not found yet >=   0.000000000e+00        (329196; 1295588)
+10982110: mip =     not found yet >=   0.000000000e+00        (329219; 1295770)
+10983566: mip =     not found yet >=   0.000000000e+00        (329235; 1295964)
+10985183: mip =     not found yet >=   0.000000000e+00        (329268; 1296129)
+10986349: mip =     not found yet >=   0.000000000e+00        (329341; 1296287)
+10987693: mip =     not found yet >=   0.000000000e+00        (329381; 1296458)
+10989097: mip =     not found yet >=   0.000000000e+00        (329456; 1296625)
+10990828: mip =     not found yet >=   0.000000000e+00        (329487; 1296794)
Time used: 19625.0 secs.  Memory used: 512.7 Mb.
+10992183: mip =     not found yet >=   0.000000000e+00     

+11113266: mip =     not found yet >=   0.000000000e+00        (332302; 1312703)
+11114727: mip =     not found yet >=   0.000000000e+00        (332338; 1312872)
+11116158: mip =     not found yet >=   0.000000000e+00        (332382; 1313029)
+11117485: mip =     not found yet >=   0.000000000e+00        (332409; 1313220)
+11118740: mip =     not found yet >=   0.000000000e+00        (332453; 1313388)
+11120601: mip =     not found yet >=   0.000000000e+00        (332570; 1313540)
+11122053: mip =     not found yet >=   0.000000000e+00        (332620; 1313709)
+11123772: mip =     not found yet >=   0.000000000e+00        (332723; 1313867)
+11125106: mip =     not found yet >=   0.000000000e+00        (332797; 1314031)
Time used: 20105.2 secs.  Memory used: 518.4 Mb.
+11126717: mip =     not found yet >=   0.000000000e+00        (332905; 1314175)
+11128231: mip =     not found yet >=   0.000000000e+00        (332922; 1314376)
+11129517: mip =     not found yet >=   0.000000000e+00     

+11251453: mip =     not found yet >=   0.000000000e+00        (335506; 1330351)
+11253266: mip =     not found yet >=   0.000000000e+00        (335526; 1330531)
+11254954: mip =     not found yet >=   0.000000000e+00        (335511; 1330770)
+11256957: mip =     not found yet >=   0.000000000e+00        (335524; 1330964)
+11258865: mip =     not found yet >=   0.000000000e+00        (335540; 1331171)
+11260544: mip =     not found yet >=   0.000000000e+00        (335581; 1331349)
+11262736: mip =     not found yet >=   0.000000000e+00        (335689; 1331520)
+11264373: mip =     not found yet >=   0.000000000e+00        (335720; 1331696)
Time used: 20585.4 secs.  Memory used: 523.2 Mb.
+11265742: mip =     not found yet >=   0.000000000e+00        (335737; 1331894)
+11267379: mip =     not found yet >=   0.000000000e+00        (335780; 1332057)
+11269119: mip =     not found yet >=   0.000000000e+00        (335874; 1332202)
+11270566: mip =     not found yet >=   0.000000000e+00     

KeyboardInterrupt: 

In [ ]:
import random, pulp

def build_base(n, v_dict, m):
    prob = pulp.LpProblem("Base", pulp.LpMinimize)
    mu = {x: pulp.LpVariable(f"mu_{x}", lowBound=0, cat='Integer')
          for x in range(2**n)}
    d  = {β: pulp.LpVariable(f"d_{β}", lowBound=None, cat='Integer')
          for β in range(2**n)}
    T  = pulp.LpVariable("T", lowBound=0, cat='Integer')

    # total‐size
    prob += sum(mu.values()) == 2**m
#     # divisibility + odd‐d
#     for β, v in v_dict.items():
#         expr = pulp.lpSum(((-1)**(bin(x&β).count('1')%2))*mu[x]
#                           for x in mu)
#         # force d[β] odd by writing 2*d[β]+1
#         prob += expr == 2**(v+1)*(2*d[β] + 1)
        
        # divisibility + odd‐d
    for β in range(1, 2**n):
        v = v_dict.get(β, 0)
        expr = pulp.lpSum(((-1)**(bin(x&β).count('1')%2))*mu[x]
                          for x in mu)
        # force d[β] odd by writing 2*d[β]+1
        prob += expr == 2**(v+1)*(2*d[β] + 1)
        
        
    # uniformity cap
    for x in mu:
        prob += mu[x] <= T

    return prob, mu, d, T

def random_objective_enumerate(n, v_dict, m, K=5):
    sols = []
    for i in range(K):
        prob, mu, d, T = build_base(n, v_dict, m)
        # *maximize* a random linear form on mu
        prob.sense = pulp.LpMaximize
        weights = {x: random.random() for x in mu}
        prob.setObjective(pulp.lpSum(weights[x]*mu[x] for x in mu))

        status = prob.solve(pulp.GLPK_CMD(msg=False))
        if pulp.LpStatus[status] != "Optimal":
            break

        mu_sol = {x:int(mu[x].value()) for x in mu}
        d_sol  = {β:int(d[β].value())   for β in d}
        sols.append((mu_sol, d_sol))
    return sols

# --- example ---
if __name__=="__main__":
    n, m = 4, 5
    v_dict = {1:2, 2:1, 12:2}
    solutions = random_objective_enumerate(n, v_dict, m, K=5)
    for i,(μ,d) in enumerate(solutions,1):
        print(f"Solution #{i}")
        print(" mu:",μ)
        print(" d:",d)
        print()
